In [11]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from cnp.experiment import WorkingDirectory

import os
import pickle

root = '/scratches/cblgpu07/em626/kernelcnp/kernelcnp/experiments/synthetic'

In [13]:
# Directory where data are stored
data_name = 'eq-2-8-50-100-100-0'

# Path to data
data_dir = os.path.join(root, 'toy-data', data_name)
data_directory = WorkingDirectory(root=data_dir)

# Load data
file = open(data_directory.file('valid-data.pkl'), 'rb')
data = pickle.load(file)
file.close()

Root: /scratches/cblgpu07/em626/kernelcnp/kernelcnp/experiments/synthetic/toy-data/eq-2-8-50-100-100-0


In [ ]:
# Generate data
def generate_dataset(data_type, num_context, num_target, noise=5e-2):
    
    # Set up covariance
    if data_type == 'eq':
        cov = EQ().stretch(1.)

    elif data_type == 'matern':
        cov = Matern52().stretch(1.)

    elif data_type == 'noisy-mixture-slow':
        cov = EQ().stretch(1.) + EQ().stretch(0.5)

    elif data_type == 'weakly-periodic-slow':
        cov = EQ().stretch(1.) * EQ().periodic(period=0.5)
    
    # Choose line to constrain along and choose inputs
    x0 = -2.
    x1 = 2.
    y0 = 4. * np.random.rand() - 2.
    y1 = 4. * np.random.rand() - 2.
    
    x = torch.linspace(x0, x1, 100)
    y = torch.linspace(y0, y1, 100)
    
    x_context = torch.stack([x, y], axis=-1)
    
    # Sample data
    

In [ ]:
# List models 

# Select input range

# For loop over models
#   Load model
#   Plot 1D marginals or samples
#   Save